In [1]:
import os
import re
import numpy as np
import pandas as pd
import urllib.request
from tqdm import tqdm
import time

In [2]:
base_path = os.getcwd()

### BeautifulSoup

In [3]:
from bs4 import BeautifulSoup

In [116]:
html_src_file = os.path.join(base_path, 'main_src.txt')
with open(html_src_file) as f:
    html_src = f.read()

In [117]:
soup = BeautifulSoup(html_src)
org_table = soup.findAll("table", {"class": "tbl"})[0]
org_table_body = org_table.find_all('tbody')[0]
org_list = org_table_body.find_all('tr')

print(f"num orgs: {len(org_list)}")

def get_substring_btw_quotes(text):
    m = re.search(".+\'(.+?)\'", text)
    if m:
        found = m.group(1)
    else:
        raise ValueError(f'{text}')
    return found

      
organizations = []
for i, org in enumerate(org_list):
    fields = org.find_all('td')

    temp = fields[0].find('a')['href']
    temp_split = temp.split(',')

    k1 = get_substring_btw_quotes(temp_split[0])
    k2 = get_substring_btw_quotes(temp_split[1])    
    
    organizations.append({
        'name': fields[0].text,
        'type': fields[1].text,
        'department': fields[2].text,
        'k1': k1,
        'k2': k2
    })

num orgs: 363


In [182]:
start = time.time()

dfs = []
for i, org in tqdm(enumerate(organizations)):
    url = 'http://www.alio.go.kr/popReportTerm.do?apbaId={0}&reportFormRootNo={1}'.format(org['k1'], org['k2'])
    fs = urllib.request.urlopen(url)
    s = fs.read().decode('utf-8')
    fs.close()

    soup = BeautifulSoup(s)
    
    # Get all tables
    tables = soup.find_all('table')
    
    # Find the first table whose class is not nb
    table = None
    for i, t in enumerate(tables):
        try:
            t['class']
        except:
            table = t
            break
            
    # Check if the table has been found
    if table is None:
        print(f"No table found for: {org['name']}")
        continue
    
    # Extract info from the table
    columns = []
    data = []
    for i, row in enumerate(table.find_all('tr')):
        if i == 0:
            for header in row.find_all('th'):
                columns.append(header.text)

        else:
            row_data = []
            for cell in row.find_all('td'):
                row_data.append(cell.text)
            data.append(row_data)

    df = pd.DataFrame(data, columns=columns)

    df['org_name'] = org['name']
    df['org_type'] = org['type']
    df['department'] = org['department']
              
    try:
        df = df[['org_name', 'org_type', 'department', '징계처분일', '징계종류', '징계사유', '고발여부']]   
    except KeyError as e:
        print(f"Key Error while processing {org['name']}")
        print(f"Columns: {columns}")
              
    dfs.append(df)

df_combined = pd.concat(dfs)
              

end = time.time()
print(f"Elapsed time: {str(end - start)}")

No table found for: (재)APEC기후센터
No table found for: (재)예술경영지원센터
No table found for: # KDI국제정책대학원
No table found for: 과학기술일자리진흥원
No table found for: 국가생명윤리정책원
No table found for: # 녹색기술센터
No table found for: 농림식품기술기획평가원
No table found for: 민주화운동기념사업회
No table found for: 새만금개발공사
No table found for: # 서울요양원
No table found for: 수도권매립지관리공사
No table found for: 신용보증재단중앙회
No table found for: 아동권리보장원
No table found for: 에너지경제연구원
No table found for: # 육아정책연구소
No table found for: 재단법인 의료기관평가인증원
No table found for: 재단법인 한국여성과학기술인지원센터


KeyError: "['징계사유', '징계종류', '고발여부', '징계처분일'] not in index"

In [184]:
df_combined = pd.concat(dfs)

In [188]:
save_path = os.path.join(base_path, 'data', 'example.csv')

In [189]:
df_combined.to_csv(save_path)

,org_name,org_type,department,징계처분일,징계종류,징계사유,고발여부
0,(사)남북교류협력지원협회,기타공공기관,통일부,2017년 02월 24일,정직,성실의무 및 법규준수 위반,미고발
1,(사)남북교류협력지원협회,기타공공기관,통일부,2019년 05월 22일,감봉,성실의무 및 법규준수 위반,미고발
2,(사)남북교류협력지원협회,기타공공기관,통일부,2019년 05월 22일,감봉,성실의무 및 법규준수 위반,미고발
0,(재)국제원산지정보원,기타공공기관,관세청,2020년 06월 10일,감봉,복무의무 위반 등,미고발
0,(재)우체국금융개발원,준정부기관(위탁집행형),과학기술정보통신부,2018년 01월 09일,견책,폭언 및 가혹행위,미고발
...,...,...,...,...,...,...,...
16,한국과학기술정보연구원,기타공공기관,과학기술정보통신부,2016년 11월 07일,해임,품위유지의 의무 등 위반,미고발
0,# 한국과학영재학교,부설기관(한국과학기술원),과학기술정보통신부,2018년 02월 22일,견책,채용전형 관리부실 및 평가 불공정,미고발
1,# 한국과학영재학교,부설기관(한국과학기술원),과학기술정보통신부,2018년 02월 22일,견책,채용전형 관리부실 및 평가 불공정,미고발
2,# 한국과학영재학교,부설기관(한국과학기술원),과학기술정보통신부,2019년 11월 12일,감봉,발전기금 과실금 집행 부적정,미고발


In [ ]:
# urllib를 이용하기 위해 urllib.request를 import 해줘야한다.
import urllib.request

# urllib.request.urlopen(url)에서 url에는 데이터 추출을 원하는 웹 주소를 넣으면 된다.
# 이 함수는 HTTPResponse를 반환해주는데, 이는 파일 스트림과 비슷한 역할을 한다고 보면 된다.
# 본 소스코드에서는 'Python Pip 설치' 포스트 주소를 사용하였다.


# urlopen의 리턴값을 이용해 데이터(html 코드)를 읽어오는 방법은 크게 3가지가 있는데
# read(), readline(), readlines()이다.

# read()는 모든 데이터를 하나의 문자열 형태로 반환해주고
# readline()은 데이터의 한 줄만 읽어온다.(개행문자가 나올 때 까지)
# readlines()는 모든 데이터를 개행문자('\n')를 기준으로 잘라내어 리스트 형식으로 반환한다.

# read() 사용 예
# decode에 대해서는 아래에서 설명하겠다.
s = fs.read().decode('utf-8')
print(s)

# fs를 닫아준다.
fs.close()
[출처] Python을 이용하여 웹 스크래핑하기|작성자 Black Stone

### Selenium

In [10]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys

# # Import webdriver for automation
# from selenium import webdriver

# # Importing the ChromeDriver class for handling webdriver
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(ChromeDriverManager().install()) # diver

# driver.get("http://www.python.org")


# assert "Python" in driver.title
# elem = driver.find_element_by_name("q")
# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
# driver.close()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/bang/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache
